# Steam Game Recommender


In [1]:
import pandas as pd

## Data Preparation

## Steam Video Games
This dataset has been sourced from [Kaggle Datasets](https://www.kaggle.com/datasets). This [dataset](https://www.kaggle.com/tamber/steam-video-games) was created by [Tamber](https://tamber.com/) and was generated entirely from public Steam Data.

### Data Content
This dataset is a list of user behaviors, with columns: user-id, game-title, behavior-name, value. The behaviors included are 'purchase' and 'play'. The value indicates the degree to which the behavior was performed - in the case of 'purchase' the value is always 1, and in the case of 'play' the value represents the number of hours the user has played the game.

In [2]:
rawdata = pd.read_csv('data/steam-200k.csv',header=None,names=['user-id','game-title','behavior-name','hours','null'])

In [3]:
rawdata[:5]

,user-id,game-title,behavior-name,hours,null
0,151603712,The Elder Scrolls V Skyrim,purchase,1.0,0
1,151603712,The Elder Scrolls V Skyrim,play,273.0,0
2,151603712,Fallout 4,purchase,1.0,0
3,151603712,Fallout 4,play,87.0,0
4,151603712,Spore,purchase,1.0,0


Since only playtime data is required, the rows which have the 'purchase' behavior are removed.

In [4]:
playtimedata=rawdata[rawdata['behavior-name']=='play']
playtimedata.drop(['behavior-name','null'],axis=1,inplace=True)

/home/rahulhp/anaconda2/envs/gl-env/lib/python2.7/site-packages/ipykernel/__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app


In [5]:
playtimedata[:5]

,user-id,game-title,hours
1,151603712,The Elder Scrolls V Skyrim,273.0
3,151603712,Fallout 4,87.0
5,151603712,Spore,14.9
7,151603712,Fallout New Vegas,12.1
9,151603712,Left 4 Dead 2,8.9


## Exploratory Data Analysis

### User Data

We find the distribution of games played by each user.

In [6]:
userdata=playtimedata[['user-id','game-title']].groupby('user-id').count().reset_index().rename(columns={'game-title':'games-played'}).sort_values(by='games-played',ascending=False)

In [7]:
userdata[:5]

,user-id,games-played
1478,62990992,498
175,11403772,314
4506,138941587,299
989,47457723,298
1070,49893565,297


In [8]:
len(userdata)

11350

There are 11350 unique users in this dataset. A huge majority of the players (57%) play only one game. 80% of the players play less than 7 games.

### Game Data

We find the distribution of players each game has.

In [9]:
gamedata=playtimedata[['user-id','game-title']].groupby('game-title').count().reset_index().rename(columns={'user-id':'players'}).sort_values('players',ascending=False)

In [10]:
gamedata[:10]

,game-title,players
922,Dota 2,4841
2994,Team Fortress 2,2323
673,Counter-Strike Global Offensive,1377
3350,Unturned,1069
1733,Left 4 Dead 2,801
675,Counter-Strike Source,715
3067,The Elder Scrolls V Skyrim,677
1313,Garry's Mod,666
670,Counter-Strike,568
2691,Sid Meier's Civilization V,554


In [11]:
len(gamedata)

3600

There are 3600 unique games in the dataset. DOTA 2 is clearly the most popular game having more than twice the number of players as it's nearest competitor Team Fortress 2. The games are very unevenly distributed.

In [12]:
for threshold in [1,2,5,10,50,100,200,500,1000]:
    print 'Number of games which have more than',threshold,'players:',len(gamedata[gamedata['players']>threshold])

Number of games which have more than 1 players: 2582
Number of games which have more than 2 players: 2118
Number of games which have more than 5 players: 1438
Number of games which have more than 10 players: 989
Number of games which have more than 50 players: 284
Number of games which have more than 100 players: 133
Number of games which have more than 200 players: 40
Number of games which have more than 500 players: 10
Number of games which have more than 1000 players: 4


## Adding genre and developer information

Instead of using DBPedia as planned in the original capstone proposal, the [Internet Game Database API](https://www.igdb.com/api) was used since it contained detailed game information and had a good [API](https://market.mashape.com/igdbcom/internet-game-database) as compared to DBPedia which would have heavy SPARQL queries.

In [13]:
games = list(gamedata['game-title'])

In [16]:
import requests
import sys
import pickle

Since there are 3600 games and the API had a rate limitation of 7000 queries per day, the following code which queries the API for genre + developer information has been commented out. The results have been stored in a pickle file for latter usage.
The API key has also been removed for privacy reasons. The API key can be obtained from the [Mashape website](https://market.mashape.com/igdbcom/internet-game-database)

In [15]:
def get_game_data(search_string):
    API_URL = "https://igdbcom-internet-game-database-v1.p.mashape.com/games/?"
    FIELDS = "fields=name%2Cslug%2Cdevelopers%2Cgenres&limit=1&offset=0&search="
    
    search_url = API_URL + FIELDS + search_string
    headers={
        "X-Mashape-Key": "INSERT-KEY-HERE",
        "Accept": "application/json"
      }
    try:
        r = requests.get(search_url,headers=headers)
        d = dict()
        d['name'] = search_string
        d['developers'] = r.json()[0].get(['developers'][0],0)
        d['genres'] = r.json()[0].get(['genres'][0],0)
        return d
    except:
        print "\tUnexpected error: "+str(sys.exc_info()[0])
        return None

In [17]:
"""
dictlist = []
count=0
for i in games:
    print count,'\t',i
    d = get_game_data(i)
    if d is not None:
        dictlist.append(d)
    count+=1
pickle.dump(dictlist,open("gameinfo.p","wb"))    
"""

'\ndictlist = []\ncount=0\nfor i in games:\n    print count,\'\t\',i\n    d = get_game_data(i)\n    if d is not None:\n        dictlist.append(d)\n    count+=1\npickle.dump(dictlist,open("gameinfo.p","wb"))    \n'

In [18]:
gameinfo = pickle.load(open("gameinfo.p","rb"))

In [19]:
len(gameinfo)

3581


Note : Information for only 3581 games was obtained from IGDB. We'll be removing the data for the other 19 games before proceeding.

**Add note about technical debt here and the reason why the 19 games are being removed at a later stage and not right now **

### Genre Information

In [26]:
genres = set()
for game in gameinfo:
    if type(game['genres'])==list:
        for i in game['genres']:
            genres.add(i)
print len(genres)

20


There are 20 distinct genres present in the dataset.

### Developer Information

In [27]:
developers = set()
for game in gameinfo:
    if type(game['developers'])==list:
        for i in game['developers']:
            developers.add(i)
print len(developers)

1858


There are 1858 distinct developers. Since there are 3600 games in the dataset, on average each developer has made 2 games. Due to the very sparse data in this field, this field has been removed from future calculations. Only the genre information will be used.

### One hot encoding for the genres

In [36]:
genreinfo = list()
for i in gameinfo:
    genreinfo.append({'genres':i['genres'],'name':i['name']})

In [37]:
dpp = pd.DataFrame(genreinfo)

In [38]:
# http://stackoverflow.com/questions/29034928/pandas-convert-a-column-of-list-to-dummies
genre_dummies = pd.get_dummies(dpp['genres'].apply(pd.Series).stack(),prefix='genre').sum(level=0)


In [39]:
game_dummy = pd.concat([dpp,genre_dummies],axis=1).drop('genres',axis=1)

In [40]:
game_dummy

,name,genre_0.0,genre_2.0,genre_4.0,genre_5.0,genre_7.0,genre_8.0,genre_9.0,genre_10.0,genre_11.0,...,genre_14.0,genre_15.0,genre_16.0,genre_24.0,genre_25.0,genre_26.0,genre_30.0,genre_31.0,genre_32.0,genre_33.0
0,10 Second Ninja,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
1,Toy Soldiers War Chest,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,Infestation Survivor Stories,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
3,Nightmares from the Deep 2 The Siren`s Call,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
4,Ultimate General Gettysburg,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
5,Unreal Tournament 2004,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,Toki Tori 2+,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0
7,Life is Hard,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
8,World of Goo,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
9,Hollywood Visionary,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0


In [41]:
game_dummy.shape

(3581, 22)

## Removing 19 games' playtime data

Note : As noted above, information for only 3581 games was obtained from IGDB. We'll be removing the data for the other 19 games before proceeding.

In [43]:
available_games = set(game_dummy['name'])
raw_games = set(gamedata['game-title'])
print list(raw_games.difference(available_games))

['Dethroned!', 'Voxelized', 'Aberoth', 'Motorbike', 'Deepworld', 'Immune', 'Breezeblox', 'Cosmophony', 'CRYENGINE', 'Unium', 'Rexaura', 'Squirreltopia', 'KWAAN', 'Burstfire', 'Batla', 'ROCKETSROCKETSROCKETS', 'Rotieer', 'Alganon', 'Jumpdrive']


In [44]:
game_dummy = game_dummy['name'].to_frame()

In [45]:
playtimedata = pd.merge(playtimedata,game_dummy,how='right',left_on='game-title',right_on='name').drop('name',axis=1)

In [50]:
playtimedata[:5]

,user-id,game-title,hours
0,151603712,The Elder Scrolls V Skyrim,273.0
1,59945701,The Elder Scrolls V Skyrim,58.0
2,92107940,The Elder Scrolls V Skyrim,110.0
3,250006052,The Elder Scrolls V Skyrim,465.0
4,11373749,The Elder Scrolls V Skyrim,220.0


Sanity check - We'll check whether the new playtimedata has data for only 3581 games or not.

In [52]:
playtimegamedata = playtimedata[['user-id','game-title']].groupby('game-title').count().reset_index()
print len(playtimegamedata)

3581


# Create scoring function

In [56]:
import graphlab as gl

In [71]:
print len(playtimedata)
playtimedata[:5]

70443


,user-id,game-title,hours
0,151603712,The Elder Scrolls V Skyrim,273.0
1,59945701,The Elder Scrolls V Skyrim,58.0
2,92107940,The Elder Scrolls V Skyrim,110.0
3,250006052,The Elder Scrolls V Skyrim,465.0
4,11373749,The Elder Scrolls V Skyrim,220.0


In [58]:
sf = gl.SFrame(playtimedata)

This non-commercial license of GraphLab Create for academic use is assigned to f2013411@goa.bits-pilani.ac.in and will expire on May 16, 2018.


[INFO] graphlab.cython.cy_server: GraphLab Create v2.1 started. Logging: /tmp/graphlab_server_1498166642.log


Since a score is required and not the ranking of games, the ground truth scores will be calculated using the entire dataset (ie both the training and the test data). For the purpose of training, only the training data will be used to find the scores. **Note:** This will negatively affect the reliability of the scores since they vary depending on the distribution. **TO DO : Explain this part better**

In [64]:
train, test = gl.recommender.util.random_split_by_user(sf,user_id='user-id',item_id='game-title')

In [65]:
print train.shape
print test.shape

(69276, 3)
(1167, 3)


Our training dataset now contains ~70,000 interactions and our test dataset has ~1,200 interactions.

In [66]:
def create_score(dataset):
    totalhours=dataset.groupby(key_columns='user-id',operations={'total_hours':gl.aggregate.SUM('hours')})
    hoursjoined = dataset.join(totalhours,on='user-id')
    hoursjoined['personal-ratio']=hoursjoined['hours']/hoursjoined['total_hours']
    hoursjoined.remove_columns(['hours','total_hours'])
    
    gamedist=dataset.groupby(key_columns='game-title',operations={'hours-avg':gl.aggregate.MEAN('hours'),
                                                           'hours-std':gl.aggregate.STD('hours')})
    
    playerdist = dataset.join(gamedist,on='game-title')
    playerdist['normalised-ratio']=(playerdist['hours']-playerdist['hours-avg'])/playerdist['hours-std']
    playerdist.remove_columns(['hours','hours-std','hours-avg'])
    
    joined = playerdist.join(hoursjoined)
    joined['score']=joined['normalised-ratio']*joined['personal-ratio']
    joined.remove_columns(['normalised-ratio','personal-ratio'])
    return joined

In [68]:
train_score = create_score(train)

In [105]:
print "There are " + str(train.shape[0]) + " interactions in this dataset."
totalhours=train.groupby(key_columns='user-id',operations={'total_hours':gl.aggregate.SUM('hours')})
print "There are " + str(totalhours.shape[0]) + " unique users in this dataset."

hoursjoined = train.join(totalhours,on='user-id')
hoursjoined['personal-ratio']=hoursjoined['hours']/hoursjoined['total_hours']
hoursjoined = hoursjoined.remove_columns(['hours','total_hours'])
print "There are " + str(hoursjoined.shape[0]) + " interactions in the personal-ratio dataset."

gamedist=train.groupby(key_columns='game-title',operations={'hours-avg':gl.aggregate.MEAN('hours'),
                                                       'hours-std':gl.aggregate.STD('hours')})
print "There are " + str(gamedist.shape[0]) + " unique games in this dataset."

playerdist = train.join(gamedist,on='game-title')
playerdist['normalised-ratio']=(playerdist['hours']-playerdist['hours-avg'])/playerdist['hours-std']
playerdist = playerdist.remove_columns(['hours','hours-std','hours-avg'])
print "There are " + str(playerdist.shape[0]) + " interactions in the player-distribution dataset."

There are 69276 interactions in this dataset.
There are 11231 unique users in this dataset.
There are 69276 interactions in the personal-ratio dataset.
There are 3558 unique games in this dataset.
There are 69276 interactions in the player-distribution dataset.


In [108]:
type(hoursjoined)

graphlab.data_structures.sframe.SFrame

In [109]:
joined = playerdist.join(hoursjoined)

In [151]:
len(joined)

69300

In [124]:
sum(joined['personal-ratio']==0)

0

In [133]:
tcount = sf.groupby(key_columns=['user-id','game-title'],operations={'count':gl.aggregate.COUNT})

In [136]:
tcount[tcount['count']!=1]

game-title,user-id,count
Grand Theft Auto SanAndreas ...,148362155,2
Sid Meier's CivilizationIV ...,176261926,2
Grand Theft Auto III,71411882,2
Tom Clancy's H.A.W.X. 2,59925638,2
Sid Meier's CivilizationIV ...,33865373,2
Grand Theft Auto ViceCity ...,28472068,2
Grand Theft Auto SanAndreas ...,50769696,2
Sid Meier's CivilizationIV Beyond the Sword ...,176261926,2
Grand Theft Auto SanAndreas ...,28472068,2
Grand Theft Auto SanAndreas ...,71510748,2


In [147]:
a=sf[sf['user-id']==71411882]
a.materialize()
b = a[a['game-title']=='Grand Theft Auto III']
b.materialize()
b

user-id,game-title,hours
71411882,Grand Theft Auto III,1.1
71411882,Grand Theft Auto III,0.2


In [149]:
playtimedata[playtimedata['user-id']==71411882]

,user-id,game-title,hours
1444,71411882,Left 4 Dead 2,41.0
2273,71411882,Left 4 Dead,23.0
5561,71411882,Grand Theft Auto IV,1.9
13840,71411882,Counter-Strike Global Offensive,11.0
15572,71411882,Saints Row The Third,37.0
15765,71411882,Far Cry 3,37.0
16267,71411882,Just Cause 2,1.8
17688,71411882,Sleeping Dogs,39.0
19169,71411882,Metro 2033,1.5
25167,71411882,Need for Speed Hot Pursuit,4.3


**Sanity Check - The number of rows in train_score should be equal to 69276!**

In [42]:
whole_score

user-id,game-title,score
151603712,The Elder Scrolls VSkyrim ...,0.602972987509
151603712,Fallout 4,0.0584825938977
151603712,Spore,-0.00662366400323
151603712,Fallout New Vegas,-0.014898430262
151603712,Left 4 Dead 2,-0.00452930125938
151603712,HuniePop,-0.0138947174626
151603712,Path of Exile,-0.00546418978501
151603712,Poly Bridge,0.00944846084584
151603712,Left 4 Dead,-0.00273919947097
151603712,Team Fortress 2,-0.00147755824966


In [46]:
test_score=test.join(whole_score,how='left')

In [47]:
test_score.remove_column('hours')

user-id,game-title,score
53875128,S.T.A.L.K.E.R. Call ofPripyat ...,0.0447560100448
53875128,Space Pirates and Zombies,-0.00117634525096
53875128,Just Cause 2,0.00180112242911
53875128,Mad Max,0.00565466011627
53875128,Deus Ex Human Revolution,-0.00232928075576
53875128,RUNNING WITH RIFLES,0.0175782914292
53875128,Sid Meier's CivilizationIII Complete ...,-0.00231519118359
53875128,DiRT 3,-0.000681846827732
53875128,Bastion,0.00409157937528
53875128,Far Cry 3 Blood Dragon,-0.000118417439896


## Benchmark model

In [48]:
model = gl.recommender.popularity_recommender.create(train_score,user_id='user-id',item_id='game-title',target='score')

Recsys training: model = popularity

Preparing data set.

Data has 69453 observations with 11237 users and 3582 items.

Data prepared in: 0.300135s

69453 observations to process; with 3582 unique items.

In [176]:
model.show()

Canvas is accessible via web browser at the URL: http://localhost:37249/index.html
Opening Canvas in default web browser.


In [49]:
ans = model.predict(test_score)

In [50]:
ans=ans.fillna(0)

In [51]:
gl.evaluation.rmse(ans.fillna(0), test_score['score'].fillna(0))

0.40073619644715613

# Appendix
## Appendix A - Explain Scoring Function w/ code and formulae
## Appendix B - Genre wise popularity

## Appendix - Explain debugging issue with merge and multiple player-game data

**Discovery** - While creating the scoring function, it was observed that the dataset created after an inner merge has more rows than the component datasets.
The personal-ratio dataset had 69276 interactions and the playtime-distribution dataset also had 69276 interactions. It was expected that the merged dataset would also have 69276 rows. But it was observed that the merged dataset had 69300 rows. This [SO question](https://stackoverflow.com/questions/41580249/pandas-merged-inner-join-data-frame-has-more-rows-than-the-original-ones) alerted me to the fact that there might be duplicates in the dataset. I observed that certain player-game combinations had 2 playtime entries instead of just 1 (as I had assumed). These duplicate rows were resulting in the issue observed above.

For example, the user ID 71411882 and the game 'Grand Theft Auto III' had 2 entries. This issue was present in the raw data itself.

In [154]:
rawdata = pd.read_csv('data/steam-200k.csv',header=None,names=['user-id','game-title','behavior-name','hours','null'])
playtimedata=rawdata[rawdata['behavior-name']=='play']
playtimedata.drop(['behavior-name','null'],axis=1,inplace=True)

/home/rahulhp/anaconda2/envs/gl-env/lib/python2.7/site-packages/ipykernel/__main__.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()


In [166]:
combinations = playtimedata.groupby(['user-id','game-title']).count().reset_index().sort_values(by='hours',ascending=False)
print combinations[combinations['hours']>1]

         user-id                                    game-title  hours
27944   71510748                  Grand Theft Auto San Andreas      2
8827    28472068                          Grand Theft Auto III      2
58055  176261926  Sid Meier's Civilization IV Beyond the Sword      2
18414   50769696                  Grand Theft Auto San Andreas      2
43939  118664413                  Grand Theft Auto San Andreas      2
11066   33865373                   Sid Meier's Civilization IV      2
51079  148362155                  Grand Theft Auto San Andreas      2
8830    28472068                    Grand Theft Auto Vice City      2
8829    28472068                  Grand Theft Auto San Andreas      2
58054  176261926                   Sid Meier's Civilization IV      2
27873   71411882                          Grand Theft Auto III      2
22653   59925638                       Tom Clancy's H.A.W.X. 2      2
